In [11]:
!pip install mlxtend

  Attempting uninstall: joblib
    Found existing installation: joblib 1.0.1
    Uninstalling joblib-1.0.1:
      Successfully uninstalled joblib-1.0.1
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1


In [3]:
import numpy as np 
import tensorflow as tf
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from sklearn.model_selection import train_test_split


In [4]:
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.cm as cm
from matplotlib import rcParams
from collections import Counter
from nltk.tokenize import RegexpTokenizer
import re
import string
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

In [5]:
df=pd.read_csv('training.1600000.processed.noemoticon.csv')

In [6]:
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [7]:
df.rename(columns={'0':'label','1467810369':'time','Mon Apr 06 22:19:45 PDT 2009':'date','NO_QUERY':'query','_TheSpecialOne_':'username'},inplace=True)

In [8]:
df.head()

,label,time,date,query,username,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [9]:


df['text'] = df.iloc[:, -1]


In [10]:

df.drop(df.columns[5],axis=1,inplace=True)

In [11]:
df.head()

,label,time,date,query,username,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [12]:
df.dtypes


label        int64
time         int64
date        object
query       object
username    object
text        object
dtype: object

In [13]:
df.isnull().sum()

label       0
time        0
date        0
query       0
username    0
text        0
dtype: int64

In [14]:
for columns in df.columns:
    print(columns,":")
    print(df[columns].value_counts().count())

label :
2
time :
1598314
date :
774362
query :
1
username :
659775
text :
1581465


In [15]:
df['label'].value_counts()

4    800000
0    799999
Name: label, dtype: int64

In [16]:
data=df[['label','text']]

In [17]:
data.head()

,label,text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [18]:
data.loc[data['label']==4,'label']=1

C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [19]:
data['label'].value_counts()

1    800000
0    799999
Name: label, dtype: int64

In [20]:
pos=data[data['label']==1]
neg=data[data['label']==0]

In [21]:
data=pd.concat([pos,neg])

In [22]:
data.head()

,label,text
799999,1,I LOVE @Health4UandPets u guys r the best!!
800000,1,im meeting up with one of my besties tonight! ...
800001,1,"@DaRealSunisaKim Thanks for the Twitter add, S..."
800002,1,Being sick can be really cheap when it hurts t...
800003,1,@LovesBrooklyn2 he has that effect on everyone


In [23]:
        import nltk
        nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
stopwords_list=stopwords.words('english')

In [25]:
from nltk.corpus import stopwords
",".join (stopwords.words('english'))

"i,me,my,myself,we,our,ours,ourselves,you,you're,you've,you'll,you'd,your,yours,yourself,yourselves,he,him,his,himself,she,she's,her,hers,herself,it,it's,its,itself,they,them,their,theirs,themselves,what,which,who,whom,this,that,that'll,these,those,am,is,are,was,were,be,been,being,have,has,had,having,do,does,did,doing,a,an,the,and,but,if,or,because,as,until,while,of,at,by,for,with,about,against,between,into,through,during,before,after,above,below,to,from,up,down,in,out,on,off,over,under,again,further,then,once,here,there,when,where,why,how,all,any,both,each,few,more,most,other,some,such,no,nor,not,only,own,same,so,than,too,very,s,t,can,will,just,don,don't,should,should've,now,d,ll,m,o,re,ve,y,ain,aren,aren't,couldn,couldn't,didn,didn't,doesn,doesn't,hadn,hadn't,hasn,hasn't,haven,haven't,isn,isn't,ma,mightn,mightn't,mustn,mustn't,needn,needn't,shan,shan't,shouldn,shouldn't,wasn,wasn't,weren,weren't,won,won't,wouldn,wouldn't"

In [26]:
    STOPWORDS = set(stopwords.words('english'))
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
data['text'] = data['text'].apply(lambda text: cleaning_stopwords(text))
data['text'].head()

799999              I LOVE @Health4UandPets u guys r best!!
800000    im meeting one besties tonight! Cant wait!! - ...
800001    @DaRealSunisaKim Thanks Twitter add, Sunisa! I...
800002    Being sick really cheap hurts much eat real fo...
800003                      @LovesBrooklyn2 effect everyone
Name: text, dtype: object

In [27]:
punc=string.punctuation

In [28]:
punc_list=punc

In [29]:
def clean_punct(text):
    translator=str.maketrans('','',punc_list)
    return text.translate(translator)

In [30]:
data['text']=data['text'].apply(lambda text: clean_punct(text))
data.tail()

,label,text
799994,0,Sick Spending day laying bed listening taylors...
799995,0,Gmail down
799996,0,rest peace Farrah So sad
799997,0,EricUrbane Sounds like rival flagging ads Not ...
799998,0,resit exams summer wishes worked harder first ...


In [31]:
def remove_repeat(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [32]:
data['text']=data['text'].apply(lambda x: remove_repeat(x))
data.head()

,label,text
799999,1,I LOVE Health4UandPets u guys r best
800000,1,im meting one besties tonight Cant wait GIRL TALK
800001,1,DaRealSunisaKim Thanks Twiter ad Sunisa I got ...
800002,1,Being sick realy cheap hurts much eat real fod...
800003,1,LovesBroklyn2 efect everyone


In [33]:
def r_mail(text):
    return re.sub(r'\S+@\S+', '', text)
data['text']=data['text'].apply(lambda x: r_mail(x))
data.head()

,label,text
799999,1,I LOVE Health4UandPets u guys r best
800000,1,im meting one besties tonight Cant wait GIRL TALK
800001,1,DaRealSunisaKim Thanks Twiter ad Sunisa I got ...
800002,1,Being sick realy cheap hurts much eat real fod...
800003,1,LovesBroklyn2 efect everyone


In [34]:

def r_url(text):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)
data['text']=data['text'].apply(lambda x: r_url(x))
data.head()

,label,text
799999,1,I LOVE Health4UandPets u guys r best
800000,1,im meting one besties tonight Cant wait GIRL TALK
800001,1,DaRealSunisaKim Thanks Twiter ad Sunisa I got ...
800002,1,Being sick realy cheap hurts much eat real fod...
800003,1,LovesBroklyn2 efect everyone


In [35]:
def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)
data['text'] = data['text'].apply(lambda x: cleaning_numbers(x))

In [36]:
tokens=RegexpTokenizer(r'\w+')
data['text']=data['text'].apply(tokens.tokenize)

In [37]:
data.head()

,label,text
799999,1,"[I, LOVE, HealthUandPets, u, guys, r, best]"
800000,1,"[im, meting, one, besties, tonight, Cant, wait..."
800001,1,"[DaRealSunisaKim, Thanks, Twiter, ad, Sunisa, ..."
800002,1,"[Being, sick, realy, cheap, hurts, much, eat, ..."
800003,1,"[LovesBroklyn, efect, everyone]"


In [38]:
st = nltk.PorterStemmer()
def stemming_on_text(data):
    text = [st.stem(word) for word in data]
    return data

data['text']= data['text'].apply(lambda x: stemming_on_text(x))

In [39]:
data.head()

,label,text
799999,1,"[I, LOVE, HealthUandPets, u, guys, r, best]"
800000,1,"[im, meting, one, besties, tonight, Cant, wait..."
800001,1,"[DaRealSunisaKim, Thanks, Twiter, ad, Sunisa, ..."
800002,1,"[Being, sick, realy, cheap, hurts, much, eat, ..."
800003,1,"[LovesBroklyn, efect, everyone]"


In [40]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [41]:

lm = nltk.WordNetLemmatizer()
def lemmatizer_on_text(data):
    text = [lm.lemmatize(word) for word in data]
    return data

data['text'] = data['text'].apply(lambda x: lemmatizer_on_text(x))

In [42]:
x=data['text']
y=data['label']

In [44]:
max_len=500 #max number of words to keep in one sequence(tweets)
tok=Tokenizer(num_words=2000)  #max number of words to keep in vocab
tok.fit_on_texts(x) #give index number according to word frequency
seq=tok.texts_to_sequences(x) #all words converted to sequence of numbers
seq_mat=sequence.pad_sequences(seq,maxlen=max_len)

In [46]:
seq_mat.shape

(1599999, 500)

In [47]:
X_train,X_test,Y_train,Y_test=train_test_split(seq_mat,y,test_size=0.3,random_state=3)

In [49]:
def tensormodel():
    inputs=Input(name='inputs',shape=[max_len])
    layer=Embedding(2000,50,input_length=max_len)(inputs) #create vector for each word
    layer=LSTM(64)(layer) #find patterns and relationship between elements or vectors
    layer=Dense(256,name='fc1',activation='relu')(layer)
    layer=Dropout(0.5)(layer)
    layer=Dense(1,name='output',activation='sigmoid')(layer)
    model=Model(inputs=inputs,outputs=layer)
    return model

In [51]:
model=tensormodel()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [52]:
hsi=model.fit(X_train,Y_train,batch_size=90,epochs=7,validation_split=0.1)

Epoch 1/7
  236/11200 [..............................] - ETA: 2:33:43 - loss: 0.6295 - accuracy: 0.6249

KeyboardInterrupt: 

In [ ]:
acc1=model.evaluate(X_test,Y_test)

In [ ]:
y_pred = model.predict(X_test) #getting predictions on the trained model
y_pred = (y_pred > 0.5)


In [ ]:
CR=confusion_matrix(Y_test, y_pred)

fig, ax = plot_confusion_matrix(conf_mat=CR,figsize=(10, 10),
                                show_absolute=True,
                                show_normed=True,
                                colorbar=True)